In [4]:
import keras
print(keras.__version__)

SymbolAlreadyExposedError: Symbol Zeros is already exposed as ().

In [1]:
pip install --upgrade tensorflow==2.15

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1)
ERROR: No matching distribution found for tensorflow==2.15
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [8]:
from tensorflow.keras.utils import plot_model
from IPython.core.display import Image
import graphviz

# Темы:
- построить модель;
- вывести модель (структуру);
- сохранить (загрузить) модель

In [11]:
class SimpleDense(keras.Layer):
    def __init__(self, units=32):
        super().__init__()
        self.units = units

    # Create the state of the layer (weights)
    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform",
            trainable=True,
            name="kernel",
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias",
        )

    # Defines the computation
    def call(self, inputs):
        return keras.ops.matmul(inputs, self.kernel) + self.bias

In [16]:
linear_layer = SimpleDense(4)

In [17]:
print(linear_layer.weights)

[]


In [19]:
linear_layer.build((2,2))

In [20]:
print(linear_layer.weights)

[<KerasVariable shape=(2, 4), dtype=float32, path=simple_dense_3/kernel>, <KerasVariable shape=(4,), dtype=float32, path=simple_dense_3/bias>]


In [50]:
class MyModel(keras.Model):
    def __init__(self, model_size):
        super().__init__()
        self.my_layers=[keras.layers.Dense(i) for i in range(1,model_size)]
        self.my_out=keras.layers.Dense(1)

    def call(self, x):
        for layer in self.my_layers:
            x=layer(x)
        x=self.my_out(x)
        return x
    

In [51]:
model=MyModel(5)

In [5]:
x=keras.ops.convert_to_tensor([1,2,3])

AttributeError: module 'keras' has no attribute 'ops'

In [53]:
y=model(x)

TypeError: Exception encountered when calling MyModel.call().

[1m'KerasTensor' object is not callable[0m

Arguments received by MyModel.call():
  • x=tf.Tensor(shape=(3,), dtype=int32)

In [69]:
def make_model(input_shape):
    input = keras.Input(shape=input_shape)
    x=input
    for i in range(5, 9):
        x=keras.layers.Dense(i)(x)
    y=keras.layers.Dense(1)(x)
    model=keras.Model(inputs=input, outputs=y)
    return model

In [73]:
model=make_model((7,))

In [74]:
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 7)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_81 (Dense)                │ (None, 5)              │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_82 (Dense)                │ (None, 6)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_83 (Dense)                │ (None, 7)              │            49 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_84 (Dense)                │ (None, 8)              │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_85 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 198 (792.00 B)

 Trainable params: 198 (792.00 B)

 Non-trainable params: 0 (0.00 B)

Другой способ создания модели. 

In [8]:
class NeuralNetworkLinear(keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.model_layers = [keras.layers.Dense(n, activation='linear') for n in self.units]
 
    def call(self, inputs):
        x = inputs
        for layer in self.model_layers:
            x = layer(x)
        return x

In [9]:
model=NeuralNetworkLinear([5,3,1])

In [10]:
model.summary()

Model: "neural_network_linear_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
x=tf.zeros((1,5))

In [12]:
y=model(x)

In [13]:
model.summary()

Model: "neural_network_linear_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (1, 5)                 │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (1, 3)                 │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (1, 1)                 │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52 (208.00 B)

 Trainable params: 52 (208.00 B)

 Non-trainable params: 0 (0.00 B)

In [101]:
for layer in model.layers:
    print(layer.name)

dense_98
dense_99
dense_100


In [102]:
model.layers[0].get_weights()

[array([[ 0.19078463,  0.08708614, -0.32678354, -0.75267315, -0.67037535],
        [ 0.41806448,  0.74950445,  0.7548313 , -0.45623282,  0.05024487],
        [-0.5294683 ,  0.07717407,  0.38577545, -0.7295047 , -0.04694408],
        [ 0.09444684, -0.37983662,  0.15074456, -0.36912417, -0.26782835],
        [-0.61197346,  0.23110914,  0.15841919, -0.30319366, -0.15559787]],
       dtype=float32),
 array([0., 0., 0., 0., 0.], dtype=float32)]

In [4]:
class NeuralNetworkLinear(keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.model_layers = [keras.layers.Dense(n, activation='linear') for n in self.units]
 
    def call(self, inputs):
        x = inputs
        for layer in self.model_layers:
            x = layer(x)
        return x

    def make(self, input_shape):
        x=keras.layers.Input(input_shape)
        model=keras.Model(inputs=x, outputs=self.call(x))
        return model

In [5]:
model=NeuralNetworkLinear([5,3,1]).make((5,))

In [6]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            18 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52 (208.00 B)

 Trainable params: 52 (208.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
plot_model(model, to_file="model.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [8]:
pip install pydotplus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 458.9 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24552 sha256=e610c27222f74634acf6415a5e0e2ae28213bbf35ee491309420fe5879553c0f
  Stored in directory: /home/jovyan/.cache/pip/wheels/bd/ce/e8/ff9d9c699514922f57caa22fbd55b0a32761114b4c4acc9e03
Successfully built pydotplus
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip list

Package                       Version
----------------------------- ------------
absl-py                       2.1.0
alembic                       1.12.0
anyio                         4.0.0
argon2-cffi                   23.1.0
argon2-cffi-bindings          21.2.0
arrow                         1.3.0
asttokens                     2.4.0
astunparse                    1.6.3
async-generator               1.10
async-lru                     2.0.4
attrs                         23.1.0
Babel                         2.13.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
beautifulsoup4                4.12.2
bleach                        6.1.0
blinker                       1.6.3
boltons                       23.0.0
Brotli                        1.1.0
cached-property               1.5.2
certifi                       2023.7.22
certipy                       0.1.3
cffi                          1.16.0
charset-normalizer            3.3.0
colorama                      0.4.6
comm    

In [15]:
!pip install graphviz

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'